## **Code's Purpose**

This code was written simply as a fun project -> a thought experiment of analysing what
would happen if thresholding and mapping was recursively applied on the pixel data of images.

It works by essentially splitting the range 0 - 255 (0 - 256 exluding the upper limit) into
bounded segments whereby they will be mapped to one number or another depending on which numbers they
are smaller/greater than. This is really best illustrated through running the code and analysing both 
the direct code output as well as the images produced.

While initially thought to be nothing special, through experimentation with the manner of splitting the 
segments (through the use of fractional components), as well as the number of recursions; I witness
that this simple algorithm can result in images of varying natures to be created. These images can 
be 'artsy' / nostalgic / terrifying and more! Such a filter could also have possible steganographic 
implications and be used to hide/reveal information hidden in images or even just to enhance images to 
a specific nature.


## **Comparison of Images**

Original Flatiron Building Photo - https://i.pinimg.com/736x/86/1c/67/861c67c9ac9287137ef79f04b72e8f0d--the-flatiron-flatiron-building.jpg

**Altered Picture - loops = 2, frac_bound = 0.8**

![Altered Flatiron Photo](flatiron_altered_2_0.8.jpg "Altered, Artsy Image")

*More like a painting, with rosy clouds and a greater contrast of light and shadow.*

------------------------------------------------------------------

Original Face Picture - https://t3.ftcdn.net/jpg/06/53/03/80/360_F_653038009_MUnQU4ccQboqO5iitQUy1XTgpl2m05O4.jpg

**Altered Picture - loops = 1, frac_bound = 0.6**

![Altered Face Pic](face_altered_1_0.6.jpg "Altered, Creepier Image")

*More like an old image - akin to that of analog horror; with an odd shape (with eyes?) above **(her)** right shoulder.* 

*Applicable for steganographic purposed (hiding words/shapes in images that can only be shown via such methods).*

In [ ]:
# Necessary Packages for use
import numpy as np 
import cv2 
from pathlib import Path
import os
from fractions import Fraction
from decimal import Decimal

In [ ]:
## 4 main variables for user to amend in descending order of priority

img_path = None # image path to work on
loops = 2 # Positive Integers from 0 to 5 (following which there is no real impact on the splits)
frac_bound = 0.5 # decimal between 0 and 1 (non-inclusive of endpoints)
dim_limit = 600 # limit of width/height (if image has height/width greater than this, will be resized to preserve og ratio of image)
# dim_limit can be increased, tho this size is recommended and deemed suitable enough for most images

assert os.path.exists(img_path) # checks if path exists, if not --> change address and prevents rest of code from running

In [ ]:
def recurse(lower,upper,loops = 0, thresh = 0.5): # algorithm that creates the bounds
    if thresh>=1 or thresh<=0:
        raise ValueError("Split must be a positive value between 0 and 1 non-inclusive")
    if loops<=0:
        return lower,upper 
    else:
        fraction = Fraction(Decimal(str(thresh)))
        num,denom = fraction.numerator, fraction.denominator
        mid = (upper * num + lower * (denom - num))//denom
        return (recurse(lower,mid,loops-1),recurse(mid,upper,loops-1))
    
t = recurse(0,256,loops, frac_bound)
arr = list(np.array(t).flatten())
bounds = [(arr[i]+arr[i+1])//2 for i in range(0,len(arr),2)]


img = cv2.imread(img_path)
lower = 0 
h,w,c = img.shape 

if h>dim_limit or w>dim_limit: # code that resized the image accordingly
    if h>w:
        ratio = round((h/dim_limit),1) 
        h,w = int(h/ratio),int(w/ratio)
    else:
        ratio = round((w/dim_limit),1) 
        h,w = int(h/ratio),int(w/ratio)
img = cv2.resize(img,(w,h))
og_shape = img.shape
print(bounds)

In [ ]:
arr = list() # code responsible for mapping pixel values to specific boundaries
for i in img.flatten():
    for j,k in enumerate(bounds):
        if i<=k and j==0:
            arr.append(0)
            break
        elif i<=k and j!=0:
            arr.append(bounds[j-1])
            break
        elif i>=k and j==len(bounds)-1:
            arr.append(bounds[j])
            break

new_img = (np.array(arr)).reshape(og_shape)
new_img = new_img.astype(np.uint8)

In [ ]:
filename = Path(img_path).stem
_,ext = os.path.splitext(img_path)
cv2.imwrite(f"{filename}_altered_{loops}_{frac_bound}{ext}",new_img) # created image is saved in current directory

In [ ]:
cv2.imshow('New Image',new_img) # image is displayed for the viewer
cv2.waitKey(0)
cv2.destroyAllWindows()